In [4]:
import csv
import pandas as pd
import dask.dataframe as dd
import os

In [13]:
path = os.getcwd().replace( "scripts" , "data/") 
path

'/home/afhuertas/data-science/master/fall2019/ids/project/data/'

In [14]:
tp = dd.read_csv(path + 'title.principals.tsv', sep = '\t')
tp = tp[['tconst', 'nconst']]
names = dd.read_csv(path + 'name.basics.tsv', sep = '\t')
names = names[['nconst', 'primaryName']]

In [15]:
titles = pd.read_csv(path + 'title.basics.tsv', sep = '\t', error_bad_lines = False)
titles.head()
titles = titles[['tconst', 'primaryTitle', 'originalTitle', 'startYear']]
titles['tfix'] = titles['primaryTitle'].str.lower()
titles['tfix'] = titles['tfix'].str.replace('[^\w\s]','')

/home/afhuertas/data-science/env-sonnet/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
wiki = pd.read_csv(path + 'wiki_movie_plots_deduped.csv')
wiki = wiki[['Release Year', 'Title', 'Plot']]
wiki['tfix'] = wiki['Title'].str.lower()
wiki['tfix'] = wiki['tfix'].str.replace('[^\w\s]','')
wiki.head()

,Release Year,Title,Plot,tfix
0,1901,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",kansas saloon smashers
1,1901,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",love by the light of the moon
2,1901,The Martyred Presidents,"The film, just over a minute long, is composed...",the martyred presidents
3,1901,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,terrible teddy the grizzly king
4,1902,Jack and the Beanstalk,The earliest known adaptation of the classic f...,jack and the beanstalk


In [17]:
titles['startYear'] = titles['startYear'].astype(str)
wiki['Release Year'] = wiki['Release Year'].astype(str)
movies_main = titles.merge(wiki, left_on=['tfix', 'startYear'], right_on=['tfix', 'Release Year'])
movies_main.head()

,tconst,primaryTitle,originalTitle,startYear,tfix,Release Year,Title,Plot
0,tt0000399,Jack and the Beanstalk,Jack and the Beanstalk,1902,jack and the beanstalk,1902,Jack and the Beanstalk,The earliest known adaptation of the classic f...
1,tt0000420,Alice in Wonderland,Alice in Wonderland,1903,alice in wonderland,1903,Alice in Wonderland,"Alice follows a large white rabbit down a ""Rab..."
2,tt0000439,The Great Train Robbery,The Great Train Robbery,1903,the great train robbery,1903,The Great Train Robbery,The film opens with two bandits breaking into ...
3,tt0000487,The Great Train Robbery,The Great Train Robbery,1903,the great train robbery,1903,The Great Train Robbery,The film opens with two bandits breaking into ...
4,tt0000488,The Land Beyond the Sunset,The Land Beyond the Sunset,1912,the land beyond the sunset,1912,The Land Beyond the Sunset,Joe is an impoverished New York newsboy who li...


In [18]:
movies_main = dd.from_pandas(movies_main, npartitions=7)
movies_main = movies_main[['tconst', 'Title','startYear','Plot']]

In [19]:
tp = tp.merge(movies_main[['tconst','Title']], on = 'tconst')

In [20]:
names['primaryName'] = names['primaryName'].str.replace(' ','') + ' '

In [21]:
tp = tp.merge(names, left_on='nconst', right_on='nconst')

In [22]:
tp = tp.groupby(tp.tconst).primaryName.sum()
tp = tp.to_frame()
movies_main = movies_main.merge(tp, on = 'tconst')

In [23]:
rev = dd.read_csv(path + 'title.ratings.tsv', sep = '\t')
movies_main = movies_main.merge(rev, on = 'tconst')

In [25]:
movies_main = movies_main.compute()

In [27]:
#os.remove(path + "Movies_Reviews.csv")
movies_main.to_csv(path + 'Movies_Reviews.csv', index = False)

In [28]:
movies_main.head()

,tconst,Title,startYear,Plot,primaryName,averageRating,numVotes
0,tt0000399,Jack and the Beanstalk,1902,The earliest known adaptation of the classic f...,GeorgeS.Fleming ThomasWhite EdwinS.Porter Jame...,6.1,600
1,tt0000420,Alice in Wonderland,1903,"Alice follows a large white rabbit down a ""Rab...",LewisCarroll MayClark CecilM.Hepworth Blair Pe...,6.3,2161
2,tt0000439,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,GilbertM.'BronchoBilly'Anderson ScottMarble Bl...,7.3,15461
3,tt0000487,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,SiegmundLubin JackFrawley EmilyLowry,7.1,352
4,tt0000488,The Land Beyond the Sunset,1912,Joe is an impoverished New York newsboy who li...,Mrs.WilliamBechtel MartinFuller WalterEdwin Do...,6.9,623
